# Investigation: Depth ConvNet

## 1. Experiment: General HP Search
Bayesian hyperparam search. Fetch data from wandb run. 

In [68]:
import pandas as pd 
import wandb
api = wandb.Api()

# Project is specified by <entity/project-name>
runs = api.runs("niware/dp_benchmark")

# add name to be able to join the dataframes afterwards
summary_list = pd.DataFrame(
    columns=["name"]+list(runs[0].summary._json_dict.keys())
).set_index("name")
config_list = pd.DataFrame(
    columns=["name"]+[
        #  We remove special values that start with _.
        key for key in runs[0].config.keys() if not key.startswith('_')
    ]
).set_index("name")
for run in runs: 
    # .summary contains the output keys/values for metrics like accuracy.
    #  We call ._json_dict to omit large files 
    summary_dict = run.summary._json_dict
    summary_dict["name"] = run.name
    summary_list = summary_list.append(
        summary_dict, 
        ignore_index = True,
    )

    # .config contains the hyperparameters.
    #  We remove special values that start with _.
    config_dict = {
        k: v for k,v in run.config.items()
        if not k.startswith('_')
    }
    config_dict["name"] = run.name
    config_list = config_list.append(
        config_dict,
        ignore_index = True,
    )

runs_df = summary_list.set_index("name").join(config_list.set_index("name"))
runs_df
#runs_df.to_csv("project.csv")

,gradients/model.fc2.bias,gradients/model.fc3.bias,gradients/model.fc1.bias,_runtime,gradients/model.fc3.weight,gradients/model.fc2.weight,gradients/model.conv_layers.9.bias,gradients/model.fc1.weight,val_acc,gradients/model.conv_layers.15.bias,...,datamodule_class,noise_multiplier,virtual_batch_size,opt_kwargs/momentum,model.noise_multiplier,opt_kwargs/weight_decay,config,model.batch_size,model.conv_layers,model.kernel_size
name,,,,,,,,,,,,,,,,,,,,,
absurd-sweep-1,NaN,NaN,NaN,17,NaN,NaN,NaN,NaN,0.096479,NaN,...,None,0.322147,0,0,0.322147,0,NaN,NaN,NaN,NaN
absurd-sweep-128,"{'_type': 'histogram', 'bins': [0, 0, 0, 0, 0,...","{'values': [1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0...","{'values': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0...",572,"{'bins': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","{'_type': 'histogram', 'bins': [0, 0, 0, 0, 0,...","{'values': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0...","{'_type': 'histogram', 'bins': [0, 0, 0, 0, 0,...",0.100000,NaN,...,None,0.637170,0,0,0.637170,0,NaN,NaN,NaN,NaN
absurd-sweep-18,NaN,NaN,NaN,6,NaN,NaN,NaN,NaN,0.100749,NaN,...,None,0.278692,0,0,0.278692,0,NaN,NaN,NaN,NaN
absurd-sweep-51,NaN,NaN,NaN,186,NaN,NaN,NaN,NaN,0.100080,NaN,...,None,2.749763,0,0,2.749763,0,NaN,NaN,NaN,NaN
absurd-sweep-54,NaN,NaN,NaN,186,NaN,NaN,NaN,NaN,0.098088,NaN,...,None,3.542142,0,0,3.542142,0,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
zany-sweep-47,NaN,NaN,NaN,187,NaN,NaN,NaN,NaN,0.096096,NaN,...,None,3.184436,0,0,3.184436,0,NaN,NaN,NaN,NaN
zany-sweep-50,NaN,NaN,NaN,183,NaN,NaN,NaN,NaN,0.107349,NaN,...,None,1.833162,0,0,1.833162,0,NaN,NaN,NaN,NaN
zesty-sweep-101,NaN,NaN,NaN,186,NaN,NaN,NaN,NaN,0.081060,NaN,...,None,2.965403,0,0,2.965403,0,NaN,NaN,NaN,NaN


In [ ]:
# reinladen und umbauen -- um danach standard abfragen zu machen 
# welche depth waren am relevantesten?
# dann 

In [77]:
# TODO: finsih this
# important cols for this experiment
important_cols = [
    "L2_clip",
    "depth",
    "noise_multiplier",
    "epoch",
    "global grad norm",
    "spent_epsilon",
    "train_loss",
    "trainer/global_step",
    "val_acc",
    "val_loss",
]
final_df = runs_df.loc[:, important_cols]
final_df.sort

,L2_clip,depth,noise_multiplier,epoch,global grad norm,spent_epsilon,train_loss,trainer/global_step,val_acc,val_loss
name,,,,,,,,,,
absurd-sweep-1,0.272357,NaN,0.322147,0,NaN,10.015257,2.311016e+00,44,0.096479,2.306946e+00
absurd-sweep-128,5.672955,4,0.637170,12,1.624411e-01,2.636813,2.296496e+00,16249,0.100000,1.004676e+03
absurd-sweep-18,3.760428,3,0.278692,0,2.435453e-01,10.313487,2.306978e+00,1,0.100749,2.304475e+00
absurd-sweep-51,1.269953,1,2.749763,0,5.403987e+09,0.106456,1.649384e+09,1249,0.100080,5.243708e+08
absurd-sweep-54,1.523124,1,3.542142,0,4.266941e+11,0.104967,2.259754e+11,1249,0.098088,7.284596e+10
...,...,...,...,...,...,...,...,...,...,...
zany-sweep-47,1.556615,1,3.184436,0,1.711065e+11,0.105493,8.078747e+10,1249,0.096096,2.674613e+10
zany-sweep-50,0.685255,NaN,1.833162,0,NaN,0.151725,4.430770e+04,1249,0.107349,1.148248e+04
zesty-sweep-101,2.684312,1,2.965403,0,1.813320e+13,0.105921,1.343531e+13,1249,0.081060,4.220907e+12


## 2. HP Search for promissing configurations

In [ ]:
# TODO: use pandas tool to determine the most promissing configs and best params
# TODO: final picture with baseline (no-dp) and optimal dp searches for all 7 depth

In [ ]:
# the ranges for dp clip and multiplier we choose as max and min over the top 10 runs 
# (that are also above 0.1) of all depth models seperately
# TODO: right down or calculate auto

# learning rate was set to 0.5 and is now set to a fixed range because of the varying magnitudes of gradients
# when considering vastly different numbers of params